# 📘 Chapter 19: Deploying TensorFlow Models to Production

## 🎯 Tujuan Bab
Bab ini membahas cara menyimpan, men-deploy, dan mengkonversi model TensorFlow untuk digunakan di dunia nyata — baik di server, mobile, maupun web.

---

## 🛠️ Metode Deployment yang Dibahas

1. **TensorFlow SavedModel**
   - Format model standar yang digunakan oleh TensorFlow Serving.
   - Mendukung signature, asset, dan variabel terpisah.

2. **.keras Format (Keras 3)**
   - Format default di Keras 3 untuk menyimpan model lengkap dalam satu file `.keras`.
   - Disarankan untuk reproducibility dan integrasi.

3. **TensorFlow Lite (TFLite)**
   - Format ringan untuk model ML di perangkat mobile (Android, iOS, microcontrollers).
   - Dikonversi menggunakan `tf.lite.TFLiteConverter`.

4. **TensorFlow.js (TF.js)**
   - Konversi model ke format web untuk dijalankan di browser menggunakan JavaScript.
   - Konversi dilakukan via CLI: `tensorflowjs_converter`.

---

## 🧠 Inti Utama

- Model harus dikemas dengan baik sebelum dikirim ke lingkungan produksi.
- Format penyimpanan memengaruhi kompatibilitas dan performa.
- Pilih format tergantung pada **target deployment**:
  - Server: SavedModel
  - Mobile: TFLite
  - Web: TF.js

---

## ✅ Kesimpulan

Chapter ini menunjukkan bahwa model machine learning tidak hanya berhenti di training, tapi harus siap untuk dipakai dalam konteks real-world melalui deployment. TensorFlow menyediakan tools lengkap untuk menjembatani kebutuhan tersebut.


In [1]:
# ✅ Import Library
import tensorflow as tf
from tensorflow import keras
import numpy as np

# ===========================================
# 🧠 Step 1: Latih Model Sederhana
# ===========================================
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_split=0.1)

# ===========================================
# 💾 Step 2: Simpan Model untuk Produksi (.keras dan SavedModel)
# ===========================================
model.save("model_production.keras")  # Format Keras 3 (modern)

# Untuk TensorFlow Serving atau TFLite: simpan dalam format SavedModel
model.export("model_production_savedmodel")  # hanya berlaku di Keras 3

# ===========================================
# 📦 Step 3: Konversi ke TFLite
# ===========================================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model berhasil dikonversi ke TFLite!")

# ===========================================
# 🌐 Step 4: Konversi ke TensorFlow.js (opsional, manual via CLI)
# ===========================================
# Jalankan secara lokal di terminal:
# !pip install tensorflowjs
# !tensorflowjs_converter --input_format keras model_production.keras tfjs_model/


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7750 - loss: 0.6414 - val_accuracy: 0.8578 - val_loss: 0.3955
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8602 - loss: 0.3903 - val_accuracy: 0.8735 - val_loss: 0.3482
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8772 - loss: 0.3415 - val_accuracy: 0.8722 - val_loss: 0.3577
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8831 - loss: 0.3209 - val_accuracy: 0.8733 - val_loss: 0.3530
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8891 - loss: 0.3000 - val_accuracy: 0.8855 - val_loss: 0.3257
Saved artifact at 'model_production_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136885083691664: TensorSpec(shape=(), dtype=tf.resource, name=